# Testing the Fluspect Leaf Reflectance model

In [1]:
# Add PATH (adapt!)
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Leaf/");
push!(LOAD_PATH, "/Volumes/cfranken/code/gitHub/LSM-SPAM/src/Utils/");

In [2]:
ENV["PATH"]

"/Applications/Julia-1.1.app/Contents/Resources/julia/bin:/Users/cfranken/anaconda3/bin:/Users/cfranken/anaconda3/bin:/Users/cfranken/anaconda3/condabin:/Users/cfranken/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/aria2/bin:/opt/X11/bin"

In [3]:
# Use FluspectMod
using FluspectMod
FluspectMod.init([collect(400:2:850); collect(860:25:2400)])

┌ Info: Recompiling stale cache file /Users/cfranken/.julia/compiled/v1.1/FluspectMod.ji for FluspectMod [top-level]
└ @ Base loading.jl:1184
┌ Warning: Module MathToolsMod with build ID 1205637371137682 is missing from the cache.
│ This may mean MathToolsMod [top-level] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:947
ERROR: LoadError: LoadError: ArgumentError: Package DocStringExtensions not found in current path:
- Run `import Pkg; Pkg.add("DocStringExtensions")` to install the DocStringExtensions package.

Stacktrace:
 [1] require(::Module, ::Symbol) at ./loading.jl:823
 [2] include at ./boot.jl:326 [inlined]
 [3] include_relative(::Module, ::String) at ./loading.jl:1038
 [4] include at ./sysimg.jl:29 [inlined]
 [5] include(::String) at /Volumes/cfranken/code/gitHub/LSM-SPAM/src/Leaf/FluspectMod.jl:1
 [6] top-level scope at none:0
 [7] include at ./boot.jl:326 [inlined]
 [8] include_relative(::Module, ::String) at ./loading.jl:1038
 [9]

ErrorException: Failed to precompile FluspectMod [top-level] to /Users/cfranken/.julia/compiled/v1.1/FluspectMod.ji.

In [4]:
# Define state vector
x = [2.,100.,10.,0.000,0.1,0.01,0.01,0.5]
"""
Definition below (needed to add as vector)
N = x[1]
Cab = x[2]
Car = x[3]
Ant = x[4]
Brown = x[5]
Cw = x[6]
Cm = x[7]
Cx = x[8]
Kcaro = (1.0-Cx).* KcaV + Cx .* KcaZ;
"""

"Definition below (needed to add as vector)\nN = x[1]\nCab = x[2]\nCar = x[3]\nAnt = x[4]\nBrown = x[5]\nCw = x[6]\nCm = x[7]\nCx = x[8]\nKcaro = (1.0-Cx).* KcaV + Cx .* KcaZ;\n"

In [12]:
# Run Fluspect:
RT,Mf,Mb = FluspectMod.fluspect(x, fqe=0.01)

([0.0433159 8.57041e-7; 0.0431909 6.6511e-7; … ; 0.121295 0.123052; 0.112263 0.113914], [2.89137e-9 2.92921e-9 … 4.19565e-10 4.17403e-10; 3.85704e-9 3.90447e-9 … 7.47418e-10 7.46491e-10; … ; 4.38939e-8 4.50724e-8 … 6.87258e-8 6.84089e-8; 1.68084e-8 1.72596e-8 … 2.63169e-8 2.61955e-8], [5.48908e-7 5.67961e-7 … 4.05275e-9 3.01284e-9; 8.2807e-7 8.56897e-7 … 7.39882e-9 5.49876e-9; … ; 7.55997e-8 7.7782e-8 … 9.36904e-8 9.0325e-8; 2.8947e-8 2.97826e-8 … 3.58747e-8 3.45861e-8])

In [6]:
using BenchmarkTools
@benchmark RT,Mf,Mb = FluspectMod.fluspect(x.+rand(1)/1000, fqe=0.01)

BenchmarkTools.Trial: 
  memory estimate:  19.27 MiB
  allocs estimate:  2645
  --------------
  minimum time:     3.022 ms (0.00% GC)
  median time:      3.914 ms (18.44% GC)
  mean time:        3.991 ms (17.79% GC)
  maximum time:     44.590 ms (92.70% GC)
  --------------
  samples:          1250
  evals/sample:     1

In [7]:
@time RT,Mf,Mb = FluspectMod.fluspect(x, fqe=0.01)

  0.010385 seconds (3.10 k allocations: 19.295 MiB, 33.81% gc time)


([0.0433159 8.57041e-7; 0.0431909 6.6511e-7; … ; 0.121295 0.123052; 0.112263 0.113914], [2.89137e-9 2.92921e-9 … 4.19565e-10 4.17403e-10; 3.85704e-9 3.90447e-9 … 7.47418e-10 7.46491e-10; … ; 4.38939e-8 4.50724e-8 … 6.87258e-8 6.84089e-8; 1.68084e-8 1.72596e-8 … 2.63169e-8 2.61955e-8], [5.48908e-7 5.67961e-7 … 4.05275e-9 3.01284e-9; 8.2807e-7 8.56897e-7 … 7.39882e-9 5.49876e-9; … ; 7.55997e-8 7.7782e-8 … 9.36904e-8 9.0325e-8; 2.8947e-8 2.97826e-8 … 3.58747e-8 3.45861e-8])

In [8]:
using Plots
plotly()

Plots.PlotlyBackend()

In [9]:
plot(FluspectMod.lambda, RT[:,1], xlabel="Wavelength", ylabel="Reflectance/1-T", label="Reflectance")
plot!(FluspectMod.lambda, 1 .-RT[:,2], xlabel="Wavelength", ylabel="Reflectance/1-T", label="1-Transmission")

In [10]:
lam = FluspectMod.lambda;
Iwle   = findall((lam.>=FluspectMod.minwle) .& (lam.<=FluspectMod.maxwle));
Iwlf   = findall((lam.>=FluspectMod.minwlf) .& (lam.<=FluspectMod.maxwlf));

In [15]:
in = 5
plot(lam[Iwlf], Mb[:,in], xlabel="Wavelength", label="SIF upward response to light at "*string(lam[Iwle[in]]))
in = 105
plot!(lam[Iwlf], Mb[:,in], xlabel="Wavelength", label="SIF upward response to light at "*string(lam[Iwle[in]]))

in = 5
plot!(lam[Iwlf], Mf[:,in], xlabel="Wavelength", label="SIF downward response to light at "*string(lam[Iwle[in]]))
in = 105
plot!(lam[Iwlf], Mf[:,in], xlabel="Wavelength", label="SIF downward response to light at "*string(lam[Iwle[in]]))


In [17]:
in = 5
plot(lam[Iwlf], Mb[:,in]+Mf[:,in], xlabel="Wavelength", label="SIF response to light at "*string(lam[Iwle[in]]))
in = 55
plot!(lam[Iwlf], Mb[:,in]+Mf[:,in], xlabel="Wavelength", label="SIF response to light at "*string(lam[Iwle[in]]))

in = 105
plot!(lam[Iwlf], Mb[:,in]+Mf[:,in], xlabel="Wavelength", label="SIF response to light at "*string(lam[Iwle[in]]))



In [18]:
M = Mb+Mf

100×150 Array{Float64,2}:
 5.51799e-7  5.7089e-7   5.82473e-7  …  5.8428e-9   4.47232e-9  3.43025e-9
 8.31927e-7  8.60801e-7  8.78299e-7     1.06478e-8  8.14624e-9  6.24525e-9
 1.17741e-6  1.21841e-6  1.24322e-6     1.82022e-8  1.39201e-8  1.06676e-8
 1.58871e-6  1.64425e-6  1.67782e-6     2.9593e-8   2.26197e-8  1.73261e-8
 2.06361e-6  2.13613e-6  2.17988e-6     4.6144e-8   3.52455e-8  2.69791e-8
 2.59835e-6  2.69026e-6  2.74558e-6  …  6.94624e-8  5.30083e-8  4.05411e-8
 3.1896e-6   3.30328e-6  3.37153e-6     1.01574e-7  7.74372e-8  5.91697e-8
 3.83739e-6  3.97526e-6  4.0578e-6      1.45233e-7  1.10633e-7  8.44678e-8
 4.54632e-6  4.71095e-6  4.80924e-6     2.04258e-7  1.55534e-7  1.18698e-7
 5.33007e-6  5.52439e-6  5.64014e-6     2.84444e-7  2.16661e-7  1.65376e-7
 6.20141e-6  6.42872e-6  6.56387e-6  …  3.93321e-7  2.9993e-7   2.29137e-7
 7.17347e-6  7.43743e-6  7.59416e-6     5.40994e-7  4.13378e-7  3.16338e-7
 8.27026e-6  8.57535e-6  8.75633e-6     7.41228e-7  5.68049e-7  4.35777e-7

In [27]:
plot(mean(M,dims=1))

UndefVarError: UndefVarError: mean not defined

In [28]:
using Statistics

In [31]:
plot(mean(M',dims=2))

In [42]:
@benchmark  r = FluspectMod.RTM_sail([x;5;1])

BenchmarkTools.Trial: 
  memory estimate:  1.96 MiB
  allocs estimate:  14676
  --------------
  minimum time:     4.114 ms (0.00% GC)
  median time:      4.478 ms (0.00% GC)
  mean time:        4.816 ms (5.90% GC)
  maximum time:     9.418 ms (36.57% GC)
  --------------
  samples:          1037
  evals/sample:     1

In [44]:
r = FluspectMod.RTM_sail([x;5;1])

287×4 Array{Float64,2}:
 0.020433   0.0128984  0.0145925  0.0125982
 0.0203892  0.0128618  0.0145498  0.012563 
 0.0203433  0.0128234  0.014505   0.012526 
 0.020293   0.0127812  0.0144559  0.0124855
 0.0202449  0.012741   0.0144091  0.0124469
 0.0201946  0.0126988  0.01436    0.0124064
 0.0201466  0.0126587  0.0143133  0.0123678
 0.0200964  0.0126167  0.0142644  0.0123274
 0.0200509  0.0125785  0.01422    0.0122908
 0.0200099  0.0125443  0.01418    0.0122579
 0.0199712  0.0125119  0.0141423  0.0122267
 0.019937   0.0124832  0.014109   0.0121992
 0.0199074  0.0124585  0.0140802  0.0121755
 ⋮                                         
 0.0754123  0.071893   0.0948142  0.0664242
 0.0817753  0.0786716  0.103691   0.072677 
 0.0865221  0.0837699  0.110364   0.0773789
 0.0905107  0.0880808  0.116003   0.081354 
 0.0953699  0.0933107  0.122799   0.086186 
 0.0973577  0.0955367  0.125747   0.0882284
 0.0882654  0.0861183  0.113783   0.0794574
 0.075148   0.0724341  0.0961589  0.0667654
 0.06611

In [49]:
plot(lam, r[:,2]./r[:,3])